# RNN

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.reset_default_graph()

In [3]:
tf.set_random_seed(777)  # reproducibility

In [4]:
sample = "How are you?"

In [30]:
#문자 사전 만들기 1단계 : char 마다 idx 부여 
#a -> 0, u -> 1 ...
idx2char = list(set(sample))  #set() : 중복제거
idx2char

['a', 'u', 'o', 'r', ' ', 'H', 'w', 'e', 'y', '?']

##### python 기초

In [31]:
for c in idx2char: #리스트를 for문(반복문)
    print(c) #c가 순차적으로 idx2char 리스트의 원소 

a
u
o
r
 
H
w
e
y
?


리스트를 이용해 for문 돌릴 때, 리스트의 원소와 함께 index도 함께 뽑고싶다면? --> enumerate

In [32]:
for i, c in enumerate(idx2char):
    print(i, c)

0 a
1 u
2 o
3 r
4  
5 H
6 w
7 e
8 y
9 ?


dictionary

In [8]:
#아래와 같이 key와 value로 이루어진 여러개의 데이터를 담는 자료구조
#key : 'name', 'job' / value : 'ardino', 'engineer'
data = {'name': 'ardino', 'job': 'engineer'}
data

{'job': 'engineer', 'name': 'ardino'}

for문을 이용해서 dictionary를 만들고 싶다면? -> comprehension(내포)

In [33]:
#문자사전만들기 2단계 : 문자 : idx dictionary 만들기
#key -> 'H', 'u' etc (data) / value -> 0, 1, 2 (index)
char2idx = {c: i for i, c in enumerate(idx2char)}  # comprehension
char2idx

{' ': 4,
 '?': 9,
 'H': 5,
 'a': 0,
 'e': 7,
 'o': 2,
 'r': 3,
 'u': 1,
 'w': 6,
 'y': 8}

#### rnn parameter

- dic size : n_input_node = X_feature의 개수  
- hidden size : n_hidden_node
- num classes : n_output_node
- sequence length : 1개 데이터의 sequence data 개수

In [10]:
dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
sequence_length = len(sample) - 1 #len(how are you?) - 1(?)

In [34]:
batch_size = 1 # data가 1개뿐 : data = {X:how are you, y: ow are you?}
learning_rate = 0.1

#### 리스트 컴프리헨션 이해하기

In [12]:
char2idx

{' ': 4,
 '?': 9,
 'H': 5,
 'a': 0,
 'e': 7,
 'o': 2,
 'r': 3,
 'u': 1,
 'w': 6,
 'y': 8}

In [13]:
for c in sample: #sample = 'how are you?'
    print(char2idx[c]) #c = h , o, w, ' ', a, r, e, ' ', y, o, u, ?
    #print(char2idx['h']) -> 2
    #print(char2idx['o']) -> 0    
    #print(char2idx['w']) -> 4    
    #print(char2idx[' ']) -> 8    

5
2
6
4
0
3
7
4
8
2
1
9


In [14]:
sample_idx = [char2idx[c] for c in sample]  # char to index
sample_idx #[h, o, w, ' ', a, r, e, ' ', y, o, u, ?] -> index

[5, 2, 6, 4, 0, 3, 7, 4, 8, 2, 1, 9]

sample_idx = 'how are you?'를 index로 변환한 값

##### 파이썬 기초 : 리스트 슬라이싱

In [15]:
data = [1, 2, 3, 4, 5, 6]
data[0:2] #0번째 ~ 1번째까지 슬라이싱

[1, 2]

In [16]:
data[:2] #0번째는 생략 가능

[1, 2]

In [17]:
data[2:] #맨끝까지 슬라이싱하고 싶다면 :뒤에도 생략하기

[3, 4, 5, 6]

In [18]:
#0번째부터 맨마지막 -1 번째까지 (맨마지막 data만 제외)시키고싶다면?
data[:-1] # -1 = 맨끝, -2 = 맨끝-1, -3 = 맨끝-2

[1, 2, 3, 4, 5]

#### sample('how are you?')로부터 X_data, y_data만들기

In [19]:
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) = how are you
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) = ow are you?

#### tensor build 시작하기

In [20]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # None = batch size -> 1
Y = tf.placeholder(tf.int32, [None, sequence_length])  # None = batch size -> 1

##### X data one_hot encoding 하기 : num_classes만큼 (n_X_feature = num_classes)

In [21]:
x_one_hot = tf.one_hot(X, num_classes)

#### 모델 구축

In [22]:
#cell 구조 선택 = BasicRNN or BasicLSTM or GRU 
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)

#cell initial_state
initial_state = cell.zero_state(batch_size, tf.float32)

#cell connect
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
#output layer = fully connected layer (cnn의 fc와 같은 개념)
X_for_fc = tf.reshape(outputs, [-1, hidden_size]) # -1 = None
hypothesis = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [24]:
# reshape out for sequence_loss
hypothesis = tf.reshape(hypothesis, [batch_size, sequence_length, num_classes])
#예측값 shape = batch size 수 x sequence 길이 x X_feature 수

#### cost & optimizer

In [25]:
#loss 계산을 위해 필요한 가중치
weights = tf.ones([batch_size, sequence_length])

#loss = sequence_loss를 이용(기본적인 loss개념은 동일)
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=hypothesis, targets=Y, weights=weights)
cost = tf.reduce_mean(sequence_loss) #avg of loss

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#### prediction

In [26]:
prediction = tf.argmax(hypothesis, axis=2)

#### Session Run

In [27]:
epochs = 30

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        #training = optimizing
        l, _ = sess.run([cost, optimizer], feed_dict={X: x_data, Y: y_data})

        if epoch%3 == 0: #epoch = 10번째마다 한번씩 test
            print('current epoch: {}/{}'.format(epoch, epochs))
            #testing = prediction
            result = sess.run(prediction, feed_dict={X: x_data})
            print('prediction_idx : {}'.format(result))
            # idx -> char 
            result_str = [idx2char[i] for i in np.squeeze(result)] #squeeze = [[1, 2, 3]] -> [1, 2, 3]

            print("cost:", l, "Prediction:", ''.join(result_str))
            print()


current epoch: 0/30
prediction_idx : [[2 2 2 4 4 4 4 4 2 4 4]]
cost: 2.3231697 Prediction: ooo     o  

current epoch: 3/30
prediction_idx : [[2 2 4 4 4 4 4 4 2 2 9]]
cost: 1.8395679 Prediction: oo      oo?

current epoch: 6/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 9 9]]
cost: 1.1696483 Prediction: ow are yo??

current epoch: 9/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.7325884 Prediction: ow are you?

current epoch: 12/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.3440708 Prediction: ow are you?

current epoch: 15/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.12636009 Prediction: ow are you?

current epoch: 18/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.046191048 Prediction: ow are you?

current epoch: 21/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.021114068 Prediction: ow are you?

current epoch: 24/30
prediction_idx : [[2 6 4 0 3 7 4 8 2 1 9]]
cost: 0.010841383 Prediction: ow are you?

current epoch: 27/30
prediction_idx : [[2 6 4 0 3 7 